## I. Explore Data

## Đọc dữ liệu và in ra một vài dòng đầu

In [1]:
import pandas as pd
import numpy as np
import os
data = pd.read_csv('../../data/data_clean.csv')
data.head(5)

,Unnamed: 0,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,0,0.420551,0.571139,0.336096,0.543891,0.680385,0.669439,0.313402,0.699753,0.286091,0
1,1,0.253300,0.297400,0.300611,0.491839,0.355661,0.719411,0.497319,0.450999,0.576793,0
2,2,0.571547,0.641311,0.321619,0.698543,0.355661,0.414652,0.562017,0.532866,0.303637,0
3,3,0.587349,0.605536,0.356244,0.603314,0.647347,0.317880,0.622089,0.808065,0.601015,0
4,4,0.643654,0.484851,0.289922,0.484900,0.514545,0.379337,0.358555,0.253606,0.496327,0


## Lựa chọn thuộc tính

In [2]:
data = data.loc[:,['ph','Hardness','Solids','Chloramines','Sulfate','Conductivity','Organic_carbon','Trihalomethanes','Turbidity','Potability']]
data.head(5)

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,0.420551,0.571139,0.336096,0.543891,0.680385,0.669439,0.313402,0.699753,0.286091,0
1,0.253300,0.297400,0.300611,0.491839,0.355661,0.719411,0.497319,0.450999,0.576793,0
2,0.571547,0.641311,0.321619,0.698543,0.355661,0.414652,0.562017,0.532866,0.303637,0
3,0.587349,0.605536,0.356244,0.603314,0.647347,0.317880,0.622089,0.808065,0.601015,0
4,0.643654,0.484851,0.289922,0.484900,0.514545,0.379337,0.358555,0.253606,0.496327,0


## OverSampling

In [3]:
# Import the library
from sklearn import datasets
from sklearn import metrics
from sklearn.model_selection import train_test_split

# Setup the data for classifier
x = data.loc[:,['ph','Hardness','Solids','Chloramines','Sulfate','Conductivity','Organic_carbon','Trihalomethanes','Turbidity']]
y = data.loc[:,['Potability']]
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
x_res, y_res = oversample.fit_resample(x, y)
print(x_res.shape,y_res.shape)

(3996, 9) (3996, 1)


## Phân chia dữ liệu (phương pháp hold out) : 70% Training , 30% Testing

In [4]:
# split data set
x_train, x_test, y_train, y_test = train_test_split(x_res, y_res, test_size=0.3)

## II. Training model

## 2.1. Single model 

### 2.1.1. KNeighborsClassifier

In [5]:
from sklearn.neighbors import KNeighborsClassifier
model_KNC = KNeighborsClassifier(n_neighbors=5)
model_KNC.fit(x_train, y_train)
print(); print(model_KNC)
# Now we have predicted the output by passing X_test and also stored real target in expected_y.
expected_y  = y_test
predicted_y_KNC = model_KNC.predict(x_test)
# Accuracy for model
from sklearn.metrics import accuracy_score 
accuracy = accuracy_score(expected_y , predicted_y_KNC)
accuracy


KNeighborsClassifier()


C:\Users\Admin\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\Admin\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


0.658882402001668

### 2.1.2. LogisticRegression

In [6]:
from sklearn.linear_model import LogisticRegression
model_LR = LogisticRegression()
model_LR.fit(x_train, y_train)
print(); print(model_LR)
# Now we have predicted the output by passing X_test and also stored real target in expected_y.
expected_y  = y_test
predicted_y_LR = model_LR.predict(x_test)
# Accuracy for model
from sklearn.metrics import accuracy_score 
accuracy = accuracy_score(expected_y , predicted_y_LR)
accuracy


LogisticRegression()


C:\Users\Admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.5104253544620517

### 2.1.3. DecisionTreeClassifier

In [7]:
from sklearn.tree import DecisionTreeClassifier
model_DT = DecisionTreeClassifier(random_state=1)
model_DT.fit(x_train, y_train)
print(); print(model_DT)
# Now we have predicted the output by passing X_test and also stored real target in expected_y.
expected_y  = y_test
predicted_y_DT = model_DT.predict(x_test)
# Accuracy for model
from sklearn.metrics import accuracy_score 
accuracy = accuracy_score(expected_y , predicted_y_DT)
accuracy


DecisionTreeClassifier(random_state=1)


0.6071726438698916

## 2.2. Basic Ensemble Model 

### 2.2.1. Max Voting 

In [8]:
from sklearn.ensemble import VotingClassifier
model_MaxVoting = VotingClassifier(estimators=[('lr', model_LR), ('kn', model_KNC),('dt', model_DT)], voting='hard')
model_MaxVoting.fit(x_train,y_train)
model_MaxVoting.score(x_test,y_test)

C:\Users\Admin\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Admin\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Admin\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to Tru

0.6255212677231026

### 2.2.2. Averaging

In [9]:
def average(x):
    for i in range(len(x)):
        if x[i] > 0.5:
            x[i] = 1
        else :
            x[i] = 0
    return x
model_average = (predicted_y_DT + predicted_y_LR + predicted_y_KNC)/3
model_average = average(model_average)
accuracy = accuracy_score(expected_y , model_average)
accuracy

0.6255212677231026

### 2.2.3. Weighted Average

In [11]:
model_wa_01 = predicted_y_KNC - 0.5
model_wa_02 = predicted_y_LR - 0.5
model_wa_03 = predicted_y_DT - 0.5
def weighted_average(x):
    for i in range(len(x)):
        if x[i] > 0:
            x[i] = 1
        else :
            x[i] = 0
    return x
model_weighted_average = (model_wa_01*0.6 + model_wa_02*0.2 + model_wa_03*0.2)
model_weighted_average = weighted_average(model_weighted_average)
accuracy = accuracy_score(expected_y , model_weighted_average)
accuracy

0.658882402001668

## 2.3. Advanced Ensemble

### 2.3.1. Bagging

In [12]:
from sklearn.ensemble import RandomForestClassifier
model_RF = RandomForestClassifier()
model_RF.fit(x_train, y_train)
print(); print(model_RF)
# Now we have predicted the output by passing X_test and also stored real target in expected_y.
expected_y  = y_test
predicted_y_RF = model_RF.predict(x_test)
# Accuracy for model
from sklearn.metrics import accuracy_score 
accuracy = accuracy_score(expected_y , predicted_y_RF)
accuracy

C:\Users\Admin\AppData\Local\Temp\ipykernel_10928\2419695806.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_RF.fit(x_train, y_train)



RandomForestClassifier()


0.7197664720600501

### 2.3.2. Boosting

In [13]:
from numpy import loadtxt
from xgboost import XGBClassifier
model_XGB = XGBClassifier()
model_XGB.fit(x_train, y_train)
print(); print(model_XGB)
# Now we have predicted the output by passing X_test and also stored real target in expected_y.
expected_y  = y_test
predicted_y_XGB = model_XGB.predict(x_test)
# Accuracy for model
from sklearn.metrics import accuracy_score 
accuracy = accuracy_score(expected_y , predicted_y_XGB)
accuracy


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0, ...)


0.6797331109257715

## III. Tối ưu mô hình

### 3.1. Tối ưu mô hình XGBoost

In [19]:
from numpy import loadtxt
from xgboost import XGBClassifier
model_XGB = XGBClassifier(n_estimators = 100000,learning_rate = 0.0001,max_depth = 15 , subsample = 0.5,random_state=42,min_child_weight=0.5,gamma=0.01,scale_pos_weight=1)
model_XGB.fit(x_train, y_train)
print(); print(model_XGB)
# Now we have predicted the output by passing X_test and also stored real target in expected_y.
expected_y  = y_test
predicted_y_XGB = model_XGB.predict(x_test)
# Accuracy for model
from sklearn.metrics import accuracy_score 
accuracy = accuracy_score(expected_y , predicted_y_XGB)
accuracy


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.01, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.0001, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=15, max_leaves=0, min_child_weight=0.5, missing=nan,
              monotone_constraints='()', n_estimators=100000, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=42, ...)


0.7281067556296914

In [15]:
# Here we have printed classification report and confusion matrix for the classifier.
print(metrics.classification_report(expected_y, predicted_y_XGB))
print(metrics.confusion_matrix(expected_y, predicted_y_XGB))

              precision    recall  f1-score   support

           0       0.72      0.71      0.72       609
           1       0.71      0.72      0.71       590

    accuracy                           0.71      1199
   macro avg       0.71      0.71      0.71      1199
weighted avg       0.71      0.71      0.71      1199

[[433 176]
 [167 423]]


### 3.2. Tối ưu mô hình Random Forest

In [21]:
from sklearn.ensemble import RandomForestClassifier
model_RF = RandomForestClassifier(n_estimators = 1000,max_depth = 15,min_samples_split=4,min_samples_leaf = 1)
model_RF.fit(x_train, y_train)
print(); print(model_RF)
# Now we have predicted the output by passing X_test and also stored real target in expected_y.
expected_y  = y_test
predicted_y_RF = model_RF.predict(x_test)
# Accuracy for model
from sklearn.metrics import accuracy_score 
accuracy = accuracy_score(expected_y , predicted_y_RF)
accuracy


C:\Users\Admin\AppData\Local\Temp\ipykernel_10928\3804496893.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_RF.fit(x_train, y_train)



RandomForestClassifier(max_depth=15, min_samples_split=4, n_estimators=1000)


0.7297748123436196

In [17]:
# Here we have printed classification report and confusion matrix for the classifier.
print(metrics.classification_report(expected_y, predicted_y_RF))
print(metrics.confusion_matrix(expected_y, predicted_y_RF))

              precision    recall  f1-score   support

           0       0.75      0.73      0.74       609
           1       0.73      0.74      0.74       590

    accuracy                           0.74      1199
   macro avg       0.74      0.74      0.74      1199
weighted avg       0.74      0.74      0.74      1199

[[444 165]
 [151 439]]
